In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.neighbors import KernelDensity
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import LeaveOneOut, train_test_split
from sklearn.base import BaseEstimator, ClassifierMixin
from scipy.stats import norm
from sklearn.datasets import load_digits

# Generative Model - Using Kernel Density Estimation

**What** is a density estimator?

A density estimator is an algorithm that takes a __D__-dimensional dataset and produces an estimate of the __D__-dimensional probability distribution which that data is drawn from
 - $P(X_1, . . .X_n)$

**Why** use KDE?

Problem with using histogram as density estimator is that the choice of bin size and locations can lead to incaccurate estimations

KDE Hyperparameters:
 - Bandwidth = 
 
KDE Runtime - KDE is known to be very computationally expensive - 

In [ ]:
cover = pd.read_csv('/Users/gabestechschulte/Downloads/covertype.csv')
cover.drop('Unnamed: 0', axis=1, inplace=True)
cover.head()

In [ ]:
X = np.array(cover.iloc[:, 0:2])
y = np.array(cover.iloc[:, -1])

In [ ]:
#X = np.array(cover.iloc[0:1000, 0:2])
#y = np.array(cover.iloc[0:1000, -1])

In [ ]:
print(X.shape)
print(y.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

### 1d Feature Example

In [ ]:
sns.kdeplot(X_train[0:2000, 0]) ## Elevation
#sns.kdeplot(X_train[:, 1]) ## Horizontal Distance to Fire Points

In [ ]:
bandwidths = 10 ** np.linspace(0.0, 1.2, 100)
grid = GridSearchCV(KernelDensity(kernel = 'gaussian'),
                   {'bandwidth': bandwidths})

grid.fit(X[0:2000, 0].reshape(-1, 1))

In [ ]:
kde = KernelDensity(bandwidth = grid.best_params_['bandwidth'],
                    kernel='gaussian').fit(X_train[0:2000, 0].reshape(-1, 1))
logprobs = kde.score_samples(X_train[0:2000, 0].reshape(-1, 1))
probs = np.exp(logprobs)
len(probs)

In [ ]:
probs.max()

## Generative Model

$P(Y = y | X)$ $\propto$ $P(X | Y)*P(Y)$

In [ ]:
class KDEClassifier(BaseEstimator, ClassifierMixin):
    
    def __init__(self, bandwidth=1.0, kernel='gaussian'):
        self.bandwidth = bandwidth
        self.kernel = kernel
    
    def fit(self, X, y):
        
        # Unique classes in the training data
        self.classes_ = np.sort(np.unique(y))
        
        # Identify observations in training set for unique classes - 5 classes means 5 training sets
        # This is used in the next steps for the likelihood function
        training_sets = [X[y == yi] for yi in self.classes_]
        
        # Fit a KDE for each unique class observation in training sets: P(X | Y) aka likelihood function
        # When "Y = y", how does the distribution of "X" change?
        self.models_ = [KernelDensity(bandwidth = self.bandwidth,
                                     kernel = self.kernel).fit(Xi)
                       for Xi in training_sets]
        
        # Prior belief based on the number of samples: P(Y) aka prior
        self.logpriors_ = [np.log(Xi.shape[0] / X.shape[0])
                          for Xi in training_sets]
        # Probs density of observing the class
        #print(np.exp(self.logpriors_))
        
        return self
    
    def predict_probability(self, X):
        
        # Score samples returns the log of the probability density function (pdf)
        # For each model, return the score on the feature vector in a transposed array
        logprobs = np.array([model.score_samples(X)
                            for model in self.models_]).T
        
        # Exponentiate since it was returned in log form
        results = np.exp(logprobs + self.logpriors_)
        
        return results / results.sum(1, keepdims = True)
    
    def predict(self, X):
        
        # Return the class that has the highest probability given the observed feature(s)
        # Posterior = P(Y | X)
        return self.classes_[np.argmax(self.predict_probability(X), 1)]

In [ ]:
classes = np.sort(np.unique(y))
training_sets = [X[y == yi] for yi in classes]
models = [KernelDensity(bandwidth = 1, kernel = 'gaussian').fit(Xi) for Xi in training_sets]
logpriors = [np.log(Xi.shape[0] / X.shape[0]) for Xi in training_sets]
logprobs = np.array([model.score_samples(X) for model in models]).T
results = np.exp(logprobs + logpriors)

$P(X | Y = y)$

When $Y = y$, how does the distribution of $X$ change? 
 - Therefore, we are looking at the change in the features distribution conditioning on the class 

In [ ]:
len(training_sets) ## 3 unique values, so 3 training sets 

In [ ]:
models ## 3 training sets, so 3 models

$P(Y)$ = Prior belief about our unique classes 

In [ ]:
np.exp(logpriors) ## probability of observing each respective class given the inputed training sets

In [ ]:
np.exp(logprobs).shape ## log of the probability density function for features using the 3 training sets

In [ ]:
classes[np.argmax(results / results.sum(1, keepdims=True))]

In [ ]:
bandwidths = 10 ** np.linspace(0, 1.5, 100)
grid = GridSearchCV(KDEClassifier(), {'bandwidth': [1]})
grid.fit(X[0:2000], y[0:2000])

In [ ]:
print(grid.best_params_)
print(grid.best_score_)